# RAG
## 3. 파이콘(이라는 서버)에 저장    

https://python.langchain.com/v0.2/docs/tutorials/rag/  
1. 문서 내용 읽기
    - https://python.langchain.com/v0.2/docs/integrations/document_loaders/
1. 문서 분할(쪼개기)
    - 문서 분할하지 않으면?
        - 토큰 수 초과로 답변을 생성하지 않을 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐
1. 임베딩 - 벡터 데이터베이스에 저장
1. 질문이 있으면 벡터 데이터베이스에 유사도 검색(코사인 유사도)
1. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1, 2. 문서 Microsoft Word 파일 읽고 분할 한번에 
loader = Docx2txtLoader("law_1.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

data_list = loader.load_and_split(text_splitter=text_splitter)

# 3. 임베딩 -> 벡터 데이터베이스에 저장 
## 환경변수 읽어오기
load_dotenv()

## 3-1. 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

## [방법3] 파이콘 서버에 저장 (두번세번하면 두번세번 같은게 추가 저장됨.)
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    documents=data_list,
    embedding=embedding, 
    index_name='law-index',     
)

## 4. 사용자 질문 + 벡터 데이터베이스에서 유사도 검색
query = "전세사기피해자 금융지원에 대해 설명해주세요."

## 벡터 데이터베이스에서 유사도 검색(기본이 4개씩)
retrieved_docs = database.similarity_search(query=query)

## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 
## 5-1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기피해 법률 전문가입니다
-[context]를 참고해 사용자의 질문에 답변해주세요. 

[context]
{retrieved_docs} 

Question : {query}

'''

formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,
)

## 5-2 LLM 모델 생성, ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model_name='gpt-4.1-nano'
)

aimessage = llm.invoke(formatted_prompt)
aimessage

c:\Users\2\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='전세사기 피해자에 대한 금융지원은 피해자의 경제적 부담을 덜어주고, 신속하게 생활 안정과 정상적인 주거 환경을 회복할 수 있도록 돕기 위한 제도입니다. 주요 내용은 다음과 같습니다.\n\n1. **전세자금대출 보증 지원**  \n   정부 또는 금융기관은 전세사기 피해자가 기존 전세금 반환을 위해 전세자금 대출을 신청할 경우, 대출 보증을 제공하여 금융기관의 대출 심사를 원활하게 하고, 피해자가 부담하는 담보 부담을 낮춥니다.\n\n2. **금융권 신용회복 지원**  \n   피해자가 금융채무를 조정하거나 신용회복 절차를 진행하는 데 있어 금융권의 협력 및 상담 지원을 통해 채무 부담을 경감하는 방안입니다.\n\n3. **재취업 및 소득지원 프로그램**  \n   피해자가 경제적으로 어려움을 겪을 경우, 정부 또는 민간기관에서 제공하는 재취업 지원 및 소득 안정화 프로그램을 통해 금융적 부담을 완화할 수 있도록 돕습니다.\n\n4. **법적 구제 절차 지원**  \n   금융기관과 협력하여 피해자가 반환 청구를 위한 법적 절차를 원활히 진행할 수 있도록 법률 상담 및 행정 지원을 제공합니다.\n\n이와 함께, 정부는 금융권과 협력하여 전세사기 피해자의 금융 부담을 완화하는 다양한 정책을 추진하고 있으니, 구체적인 금융지원 방법이나 신청 절차는 가까운 금융기관이나 법률 상담 기관에 문의하시는 것이 좋습니다. 더 상세한 맞춤형 상담이 필요하시면 언제든 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 369, 'prompt_tokens': 62, 'total_tokens': 431, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_predic